In [1]:
pip install Flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.5
    Uninstalling blinker-1.5:
      Successfully uninstalled blinker-1.5
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
#2
from flask import Flask, render_template

app = Flask(__name)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

if __name__ == '__main':
    app.run()


In [ ]:
#3
from flask import Flask, render_template

app = Flask(__name)

@app.route('/greet/<name>')
def greet(name):
    return render_template('greet.html', name=name)

if __name__ == '__main':
    app.run()


In [ ]:
#4
from flask import Flask, render_template, request

app = Flask(__name)

@app.route('/', methods=['GET', 'POST'])
def form_example():
    if request.method == 'POST':
        user_input = request.form['user_input']
        return f'You entered: {user_input}'
    return render_template('form.html')

if __name__ == '__main':
    app.run()


In [ ]:
#5
from flask import Flask, render_template, request, session, redirect, url_for

app = Flask(__name)
app.secret_key = 'your_secret_key'

@app.route('/')
def home():
    if 'username' in session:
        return f'Hello, {session["username"]}!'
    return 'Welcome, Guest!'

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        session['username'] = request.form['username']
        return redirect(url_for('home'))
    return render_template('login.html')

@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('home'))

if __name__ == '__main':
    app.run()


In [ ]:
#6
from flask import Flask, render_template, request, redirect, url_for
import os

app = Flask(__name)
app.config['UPLOAD_FOLDER'] = 'uploads'

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)

    if file:
        filename = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(filename)
        return f'File "{file.filename}" uploaded successfully.'

if __name__ == '__main__':
    app.run()


In [ ]:
#7
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///items.db'
db = SQLAlchemy(app)

class Item(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

@app.route('/')
def index():
    items = Item.query.all()
    return render_template('index.html', items=items)

@app.route('/add', methods=['POST'])
def add_item():
    if request.method == 'POST':
        name = request.form['name']
        new_item = Item(name=name)
        db.session.add(new_item)
        db.session.commit()
        return redirect(url_for('index'))

if __name__ == '__main__':
    db.create_all()
    app.run()


In [ ]:
#8
from flask import Flask, request, jsonify

app = Flask(__name)
books = []

@app.route('/books', methods=['GET'])
def get_books():
    return jsonify({'books': books})

@app.route('/books/<int:id>', methods=['GET'])
def get_book(id):
    if id < len(books):
        return jsonify({'book': books[id]})
    return 'Book not found', 404

@app.route('/books', methods=['POST'])
def create_book():
    data = request.get_json()
    books.append(data)
    return 'Book created', 201

@app.route('/books/<int:id>', methods=['PUT'])
def update_book(id):
    data = request.get_json()
    if id < len(books):
        books[id] = data
        return 'Book updated'
    return 'Book not found', 404

@app.route('/books/<int:id>', methods=['DELETE'])
def delete_book(id):
    if id < len(books):
        del books[id]
        return 'Book deleted'
    return 'Book not found', 404

if __name__ == '__main__':
    app.run()
